# Import Libraries

In [ ]:
import pytesseract
from PIL import Image
import fitz 
import numpy as np
import cv2
from matplotlib import pyplot as plt

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  

# Reading Image Paths of the PDF

In [ ]:
img1 = "C:\\Users\\marri\\Kshetra Consulting\\page1.jpg"
img2 = "C:\\Users\\marri\\Kshetra Consulting\\page2.jpg"

read_img1 = cv2.imread(img1)
read_img2 = cv2.imread(img2)

# Giving size to the image for displaying

In [ ]:
def display(im_path):
    dpi=80
    im_data = plt.imread(im_path)
    #h,w,d = im_data.shape

    if im_data.ndim == 2:  # Grayscale image
        h, w = im_data.shape
    elif im_data.ndim == 3:  # Color image
        h, w, d = im_data.shape
    else:
        raise ValueError("Unsupported image format.")

    figsize = w / float(dpi), h/ float(dpi)

    fig = plt.figure(figsize=figsize)
    ax = fig.add_axes([0,0,1,1])

    ax.axis('off')

    ax.imshow(im_data, cmap='gray' if im_data.ndim == 2 else None)
    plt.show()




display(img1)
display(img2)

# Function for recognising countours and in built preprocessing of cropped images

In [ ]:
def getContours(cvImage):
    newImage = cvImage.copy()
    
    if len(newImage.shape) == 3 and newImage.shape[2] == 3:
        # Convert to grayscale if it is a color image (3 channels)
        gray = cv2.cvtColor(newImage, cv2.COLOR_BGR2GRAY)
    elif len(newImage.shape) == 2:
        # If the image is already grayscale (1 channel), no need to convert
        gray = newImage
    else:
        raise ValueError("Unexpected image format!")
        
    blur = cv2.GaussianBlur(gray,(9,9),0)
    thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(1,2))
    dilate = cv2.dilate(thresh, kernel,iterations=2)
    cv2.imwrite("dilate.jpg",dilate)
    

    contours, temp = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours,key=lambda ctr: (cv2.boundingRect(ctr)[1], cv2.boundingRect(ctr)[0]))

    for i,c in enumerate(contours):
        x,y,w,h = cv2.boundingRect(c)
        if h>300 and w>700:
            roi_crop = newImage[y:y+h,x:x+2*h+150]
            cv2.imwrite("roi.jpg",roi_crop)
            cv2.rectangle(newImage,(x,y),(x+w,y+h),(0,255,0),2)
            roi_path = "C:\\Users\\marri\\Kshetra Consulting\\roi.jpg"
            read_roi = cv2.imread(roi_path)
            
            
            inverted1 = cv2.bitwise_not(read_roi)
            cv2.imwrite("inverted1.jpg",inverted1)
            #display("inverted1.jpg")
            
            
            def grayscale(image):
                return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            gray1 = grayscale(read_roi)
            cv2.imwrite("gray1.jpg", gray1)
            #display("gray1.jpg")
            
            
            thresh, bw1 = cv2.threshold(gray1, 180, 255, cv2. THRESH_BINARY)   #black,white
            cv2.imwrite( "bw1.jpg", bw1)
            #display ("bw1.jpg")
            
            
            def noise_removal(image):
                kernel = np.ones ((1, 1), np.uint8)
                image = cv2.dilate(image, kernel, iterations=1)
                kernel = np.ones((1, 1), np.uint8)
                image = cv2.erode(image, kernel, iterations=1)
                image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
                image = cv2.medianBlur (image, 3)
                return (image)
    
            noise1 = noise_removal(bw1)
            cv2.imwrite("noise1.jpg", noise1)
            #display("noise1.jpg")
            
            
        
            def thin_font (image):
                image = cv2. bitwise_not(image)
                kernel = np.ones ((1,1),np.uint8)
                image = cv2. erode(image, kernel, iterations=1)
                image = cv2. bitwise_not(image)
                return(image)

            eroded1 = thin_font(noise1)
            cv2.imwrite("eroded1.jpg", eroded1)
            #display("eroded1.jpg")



            def thick_font (image):
                image = cv2.bitwise_not(image)
                kernel = np.ones((1,2),np.uint8)
                image = cv2.dilate(image, kernel, iterations=1)
                image = cv2.bitwise_not(image)
                return(image)
    
            dilated1 = thick_font(noise1)
            cv2. imwrite("dilated1.jpg", dilated1)
            #display("dilated1.jpg")
            
            
            ocr_result = pytesseract.image_to_string(dilated1)
            original_string = ocr_result
            modified_string = original_string.replace("Photo", "").replace("Available", "")
            print(modified_string)
    
        cv2.imwrite("boxes.jpg",newImage)


getContours(read_img1)
getContours(read_img2)